In [6]:
# Import the required libraries
import re  # Used for regular expressions to parse lines
import pandas as pd  # Used for data manipulation and exporting to Excel

# Initialize an empty list to store variable information
variables = []
# Initialize an empty list to store lines that don't match the expected pattern
unmatched_lines = []

# Define a function to extract variable information from an R script
def extract_variable_info(r_script_path):
    try:
        # Try opening the file at the given path
        with open(r_script_path, 'r') as file:
            # Read all lines from the file
            lines = file.readlines()
    except IOError as e:
        # If there's an error opening the file, print the error
        print(f"Error reading file: {e}")
        # Return an empty DataFrame in case of file read error
        return pd.DataFrame()

    # Iterate over each line in the file
    for i, line in enumerate(lines):
        # Check if the line contains the pattern 'factor(' which indicates variable definition
        if 'factor(' in line:
            try:
                # Use regular expression to extract the variable name from the line
                variable_name = re.search(r'df\$(\w+)', line).group(1)
                # Get the line that defines the levels of the factor
                level_line = lines[i+1]
                # Get the line that defines the labels of the factor
                label_line = lines[i+2]
                # Extract the levels using regular expression
                levels = re.search(r'levels = c\((.*?)\)', level_line).group(1)
                # Extract the labels using regular expression
                labels = re.search(r'labels = c\((.*?)\)', label_line).group(1)

                # Append the extracted information to the variables list
                variables.append({
                    'Variable Name': variable_name,
                    'Level': levels,
                    'Label': labels
                })
            except (AttributeError, IndexError):
                # If there's an error in parsing, add the line to unmatched_lines
                unmatched_lines.append(line)

    # Check if there are any unmatched lines
    if unmatched_lines:
        # Open a file to log these lines
        with open('unmatched_lines.log', 'w') as log_file:
            # Write each unmatched line to the log file
            for line in unmatched_lines:
                log_file.write(line + '\n')

    # Return the extracted information as a DataFrame
    return pd.DataFrame(variables)

# Path to your R script
r_script_path = '/Users/collindougherty/Documents/Work/pipeline/backend/ncdb_recode.R'
# Call the function and store the result in a DataFrame
df = extract_variable_info(r_script_path)
# Display the DataFrame
df

# Uncomment the line below to export the DataFrame to an Excel file
#df.to_excel('variable_info.xlsx', index=False)

,Variable Name,Level,Label
0,SEX,"1, 2","""Male"", ""Female"""
